In [ ]:
import geopandas as gpd
import pandas as pd

# Part 5
## Bicycle sharing system stations - analysis and transfer learning

### Task 1

Load dataset with *venturilo* bike stations (`data/veturilo_stations.json`) and convert *lat/lon* into a geometry column. Save it to `stations_gdf` variable

In [ ]:
data_path = '../../data/veturilo_stations.json'

stations_gdf = ...

# YOUR CODE HERE
raise NotImplementedError()

stations_gdf.head()

Downloading area of Warsaw in preparation for features download. Visualization of station location on the map

In [ ]:
from srai.regionalizers import geocode_to_region_gdf

warsaw_region = geocode_to_region_gdf("Warsaw, PL")
m = warsaw_region.explore(tooltip=False, highlight=False, style_kwds={"fillOpacity": 0.3})
stations_gdf.explore(m=m, color="red")

### Task 2
Split the area of Warsaw into regions, for which we will be predicting stations location

In this example we use H3 hierachical index and split the area into hexagons of size 9 (approx 500m in diameter)

In [ ]:
from srai.plotting import plot_regions
from srai.regionalizers import H3Regionalizer

regions_gdf = ...

# YOUR CODE HERE
raise NotImplementedError()

plot_regions(regions_gdf)

### Task 3

Download the OSM tags which will be used to predict bicycle stations locations. For this case, `OSMPbfLoader` will work the best

We recommend the predefined `GEOFABRIK_LAYERS` filter, since it covers a wide range of different tags. But be honest, remove `{"shopping": "amenity=bicycle_rental"}` tag ;)

In [ ]:
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS
from srai.loaders import OSMPbfLoader

features_gdf = ...

# YOUR CODE HERE
raise NotImplementedError()

features_gdf.head()

Our features have not been associated with regions yet. We can use an *intersects* predicate and associate them with regions.

In [ ]:
from srai.joiners import IntersectionJoiner

joined_features = IntersectionJoiner().transform(regions_gdf, features_gdf)
joined_features

### Task 4

We have already accociated OSM features with regions. To train our model we have to join station locations with regions as well. Write the code which finds regions intersecting with station locations. Use those information to select positive and negative samples for classifier training (regions with and without stations). Remember that we wiil have to train model based on that, so make sure to do any neccessary undersampling to balance our training data

In [ ]:
positive_samples = ...
negative_samples = ...

# YOUR CODE HERE
raise NotImplementedError()


train_data = pd.concat([positive_samples, negative_samples])
train_data.explore("is_positive", cmap="cividis", zoom_start=13, tiles="CartoDB positron")

Let's create embeddings for each region in our city (embeddings for outside of training data will be used for visualizations). Those will serve as our *Xs* for training, and *Ys* will be binary value if station is in the area or not

In [ ]:
from srai.embedders import ContextualCountEmbedder
from srai.neighbourhoods import H3Neighbourhood

embedder = ContextualCountEmbedder(
    neighbourhood=H3Neighbourhood(),
    neighbourhood_distance=5,
    concatenate_vectors=True,
    expected_output_features=GEOFABRIK_LAYERS,
)
embeddings = embedder.transform(
    regions_gdf=regions_gdf, features_gdf=features_gdf, joint_gdf=joined_features
)
X = embeddings.loc[train_data.index].to_numpy()
Y = train_data["is_positive"].astype(int).to_numpy()

### Task 5

Select your favourite model and train a classifier for station locations

In [ ]:
from sklearn.metrics import classification_report

# YOUR CODE HERE
raise NotImplementedError()


print(classification_report(Y_test, Y_pred))

### Task 6

Run predictions for all regions and prepare visualization on the map

In [ ]:
from srai.plotting import plot_numeric_data

# YOUR CODE HERE
raise NotImplementedError()


### Final task - transfer learning

Now we have a model, which was trained on data from Warsaw. Select some other city, and run predictions on it. Let's see where to put BSS stations there

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
